## 本程序中出现并解决的一些问题：  
**1.下载mnist数据集:**  
下载成功会显示  
Extracting MNIST_data\train-images-idx3-ubyte.gz  
Extracting MNIST_data\train-labels-idx1-ubyte.gz  
Extracting MNIST_data\t10k-images-idx3-ubyte.gz  
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz 

**2.ValueError: None values not supported**  
函数中没有return  

# 分类问题

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## 下载mnist数据集

In [2]:
# number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data' , one_hot=True)

W0915 16:55:46.437091 11960 deprecation.py:323] From <ipython-input-2-8449282bccfa>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0915 16:55:46.470529 11960 deprecation.py:323] From C:\Users\24301\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0915 16:55:46.475519 11960 deprecation.py:323] From C:\Users\24301\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
I

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0915 16:55:46.815388 11960 deprecation.py:323] From C:\Users\24301\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0915 16:55:46.822374 11960 deprecation.py:323] From C:\Users\24301\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0915 16:55:46.912207 11960 deprecation.py:323] From C:\Users\24301\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


## 定义增加神经层的函数

In [3]:
#add one more layer and return the output of this layer
'''
def add_layer(inputs , in_size , out_size , activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size , out_size]))
    biases = tf.Variable(tf.zeros([1 , out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs , Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
'''

'\ndef add_layer(inputs , in_size , out_size , activation_function=None):\n    Weights = tf.Variable(tf.random_normal([in_size , out_size]))\n    biases = tf.Variable(tf.zeros([1 , out_size]) + 0.1)\n    Wx_plus_b = tf.matmul(inputs , Weights) + biases\n    if activation_function is None:\n        outputs = Wx_plus_b\n    else:\n        outputs = activation_function(Wx_plus_b)\n    return outputs\n'

## 定义函数

In [4]:
# 评价准确率的函数
def compute_accuracy(v_xs , v_ys):
    global prediction
    y_pre = sess.run(prediction , feed_dict={xs:v_xs , keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre ,1) , tf.argmax(v_ys , 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
    result = sess.run(accuracy , feed_dict={xs:v_xs , ys:v_ys , keep_prob:1})
    return result

In [5]:
# weight函数
def weight_variable(shape):
    #tf.truncated_normal(shape , mean ,stddev),stddev是标准差，神经网络中取随机数
    initial = tf.truncated_normal(shape , stddev=0.1)
    return tf.Variable(initial)

In [6]:
# bias函数
def bias_variable(shape):
    initial = tf.constant(0.1 , shape=shape)#常数，初始值是0.1
    return tf.Variable(initial)

In [7]:
# 卷积层函数
def conv2d(x , W):
    #stride[1 , x_movement , y_movement ,1]定义步长
    return tf.nn.conv2d(x , W , strides=[1,1,1,1] , padding='SAME')

In [8]:
# 池化函数
def max_pool_2x2(x):
    return tf.nn.max_pool(x , ksize=[1,2,2,1] , strides=[1,2,2,1] , padding='SAME')

## 定义输入的placeholder

In [9]:
# define placeholder for inputs to network
# 每一张图片有28*28=784个像素点 
xs = tf.placeholder(tf.float32 , [None , 784])
# 每一张图片有10个输出
ys = tf.placeholder(tf.float32 , [None , 10])

keep_prob = tf.placeholder(tf.float32)

x_image = tf.reshape(xs , [-1 , 28 , 28 , 1])
#print(x_image.shape) # [n_samples , 28 , 28 , 1]

## 添加神经层

In [10]:
# add output layer
#prediction = add_layer(xs , 784 , 10 , activation_function=tf.nn.softmax)

In [11]:
## conv1 layer ##
W_conv1 = weight_variable([5 , 5 , 1 ,32])#patch:5x5 , height/in_size(黑白/mnist只有一个通道):1 ,out_size/卷积核：32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image , W_conv1) + b_conv1)# out_size = 28x28x32
h_pool1 = max_pool_2x2(h_conv1) # out_size = 14x14x32

## conv2 layer ##
W_conv2 = weight_variable([5 , 5 , 32 , 64])#patch:5x5 , height/in_size:32 , out_size/卷积核：64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1 , W_conv2) + b_conv2)# out_size = 14x14x64
h_pool2 = max_pool_2x2(h_conv2) # out_size = 7x7x64

## func1 layer ##
W_fc1 = weight_variable([7*7*64 , 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2 , [-1 , 7*7*64])#展平
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat , W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1 , keep_prob)

## func2 layer ##
W_fc2 = weight_variable([1024 , 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop , W_fc2) + b_fc2)


W0915 16:55:47.821513 11960 deprecation.py:506] From <ipython-input-11-72a07212c759>:18: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 计算损失率(loss),softmax+cross_entropy用于分类算法

In [12]:
# the error between prediction and real data
# loss
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1]))       # loss

## 训练方式

In [13]:
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

## 创建会话

In [14]:
sess = tf.Session()

## 初始化

In [15]:
sess.run(tf.global_variables_initializer())

## 训练,逐个提取100个来进行训练，节省时间

In [16]:
for i in range(1000):
    batch_xs , batch_ys = mnist.train.next_batch(100)
    sess.run(train_step , feed_dict={xs:batch_xs , ys:batch_ys , keep_prob:1})
    if i%50 == 0:
        print(compute_accuracy(mnist.test.images , mnist.test.labels))

0.0831
0.8059
0.891
0.9209
0.9333
0.9377
0.9451
0.9484
0.9486
0.9549
0.9527
0.9633
0.9563
0.9626
0.9648
0.9695
0.9624


KeyboardInterrupt: 